In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances as compute_pairwise_distances
from scipy.spatial.distance import cdist as compute_cdist
from collections import Counter as FrequencyCounter

In [ ]:
def compute_distance_matrix(samples, centers, distance_metric):
    if distance_metric == 'euclidean':
        distance_matrix = compute_pairwise_distances(samples, centers, metric='euclidean')
    elif distance_metric == 'cosine':
        distance_matrix = compute_pairwise_distances(samples, centers, metric='cosine')
    else:
        distance_matrix = compute_cdist(samples, centers, metric='jaccard')
    return distance_matrix

def kmeans_from_scratch(samples, num_clusters, distance_metric, max_iterations=200):
    centers = samples[np.random.choice(samples.shape[0], num_clusters, replace=False)]
    iteration = 0
    previous_sse = 0
    while iteration < max_iterations:
        distance_matrix = compute_distance_matrix(samples, centers, distance_metric)
        cluster_labels = np.argmin(distance_matrix, axis=1)
        current_sse = np.sum(np.min(distance_matrix, axis=1))
        if current_sse == previous_sse or (iteration != 0 and current_sse > previous_sse):
            break
        previous_centers = centers.copy()
        for i in range(num_clusters):
            centers[i] = np.mean(samples[cluster_labels == i], axis=0)
        if np.array_equal(centers, previous_centers):
            break
        iteration += 1
        previous_sse = current_sse
    return cluster_labels, centers, current_sse, iteration


In [ ]:
samples = np.genfromtxt('data.csv', delimiter=',')
actual_labels = np.genfromtxt('label.csv', delimiter=',')
num_clusters = len(np.unique(actual_labels))

In [ ]:
labels_euclidean, _, sse_euclidean, iterations_euclidean = kmeans_from_scratch(samples, num_clusters, 'euclidean')
print('Euclidean SSE')
print(sse_euclidean)

labels_cosine, _, sse_cosine, iterations_cosine = kmeans_from_scratch(samples, num_clusters, 'cosine')
print('Cosine SSE')
print(sse_cosine)

labels_jaccard, _, sse_jaccard, iterations_jaccard = kmeans_from_scratch(samples, num_clusters, 'jaccard')
print('Jaccard SSE')
print(sse_jaccard)

Euclidean SSE
15721572.927233623
Cosine SSE
2465.07133664333
Jaccard SSE
9999.84514398036


In [ ]:
def compute_accuracy(predicted_clusters, actual_labels):
    cluster_map = {}
    for i in range(num_clusters):
        cluster_actuals = actual_labels[predicted_clusters == i]
        if len(cluster_actuals) > 0:
            label_frequency = FrequencyCounter(cluster_actuals)
            common_label = label_frequency.most_common(1)[0][0]
            cluster_map[i] = common_label
        else:
            cluster_map[i] = None
    majority_predictions = np.array([cluster_map[label] for label in predicted_clusters])
    valid_indices = majority_predictions != None
    return np.sum(majority_predictions[valid_indices] == actual_labels[valid_indices]) / len(actual_labels[valid_indices])


In [ ]:
accuracy_euclidean = compute_accuracy(labels_euclidean, actual_labels)
print('Euclidean Accuracy')
print(accuracy_euclidean * 100)

accuracy_cosine = compute_accuracy(labels_cosine, actual_labels)
print('Cosine Accuracy')
print(accuracy_cosine * 100)

accuracy_jaccard = compute_accuracy(labels_jaccard, actual_labels)
print('Jaccard Accuracy')
print(accuracy_jaccard * 100)


Euclidean Accuracy
60.07
Cosine Accuracy
61.82
Jaccard Accuracy
11.53


In [ ]:
def kmeans_centers_stable(samples, num_clusters, distance_metric, max_iterations=200):
    centers = samples[np.random.choice(samples.shape[0], num_clusters, replace=False)]
    iteration = 0
    previous_sse = 0
    while iteration < max_iterations:
        distance_matrix = compute_distance_matrix(samples, centers, distance_metric)
        cluster_labels = np.argmin(distance_matrix, axis=1)
        current_sse = np.sum(np.min(distance_matrix, axis=1))
        previous_centers = centers.copy()
        for i in range(num_clusters):
            centers[i] = np.mean(samples[cluster_labels == i], axis=0)
        if np.array_equal(centers, previous_centers):
            break
        iteration += 1
        previous_sse = current_sse
    return cluster_labels, centers, current_sse, iteration


In [ ]:
labels_euclidean, _, sse_euclidean, iterations_euclidean = kmeans_centers_stable(samples, num_clusters, 'euclidean')
print("Euclidean SSE:", sse_euclidean)
print("Euclidean Iterations:", iterations_euclidean)

labels_cosine, _, sse_cosine, iterations_cosine = kmeans_centers_stable(samples, num_clusters, 'cosine')
print("Cosine SSE:", sse_cosine)
print("Cosine Iterations:", iterations_cosine)

labels_jaccard, _, sse_jaccard, iterations_jaccard = kmeans_centers_stable(samples, num_clusters, 'jaccard')
print("Jaccard SSE:", sse_jaccard)
print("Jaccard Iterations:", iterations_jaccard)


Euclidean SSE: 15721974.152216878
Euclidean Iterations: 59
Cosine SSE: 2456.979228647383
Cosine Iterations: 70


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Jaccard SSE: 10000.0
Jaccard Iterations: 200


In [ ]:
def kmeans_stop_on_sse_increase(samples, num_clusters, distance_metric, max_iterations=200):
    centers = samples[np.random.choice(samples.shape[0], num_clusters, replace=False)]
    iteration = 0
    previous_sse = 0
    while iteration < max_iterations:
        distance_matrix = compute_distance_matrix(samples, centers, distance_metric)
        cluster_labels = np.argmin(distance_matrix, axis=1)
        current_sse = np.sum(np.min(distance_matrix, axis=1))
        if current_sse == previous_sse or (iteration != 0 and current_sse > previous_sse):
            break
        previous_centers = centers.copy()
        for i in range(num_clusters):
            centers[i] = np.mean(samples[cluster_labels == i], axis=0)
        iteration += 1
        previous_sse = current_sse
    return cluster_labels, centers, current_sse, iteration


In [ ]:
labels_euclidean, _, sse_euclidean, iterations_euclidean = kmeans_stop_on_sse_increase(samples, num_clusters, 'euclidean')
print("Euclidean SSE:", sse_euclidean)
print("Euclidean Iterations:", iterations_euclidean)

labels_cosine, _, sse_cosine, iterations_cosine = kmeans_stop_on_sse_increase(samples, num_clusters, 'cosine')
print("Cosine SSE:", sse_cosine)
print("Cosine Iterations:", iterations_cosine)

labels_jaccard, _, sse_jaccard, iterations_jaccard = kmeans_stop_on_sse_increase(samples, num_clusters, 'jaccard')
print("Jaccard SSE:", sse_jaccard)
print("Jaccard Iterations:", iterations_jaccard)


Euclidean SSE: 15650093.638524346
Euclidean Iterations: 23
Cosine SSE: 2462.9090784993705
Cosine Iterations: 38
Jaccard SSE: 9999.918944640029
Jaccard Iterations: 1


In [ ]:
def kmeans_run_full_iterations(samples, num_clusters, distance_metric, max_iterations=200):
    centers = samples[np.random.choice(samples.shape[0], num_clusters, replace=False)]
    iteration = 0
    previous_sse = 0
    while iteration < max_iterations:
        distance_matrix = compute_distance_matrix(samples, centers, distance_metric)
        cluster_labels = np.argmin(distance_matrix, axis=1)
        current_sse = np.sum(np.min(distance_matrix, axis=1))
        centers_prev = centers.copy()
        for i in range(num_clusters):
            centers[i] = np.mean(samples[cluster_labels == i], axis=0)
        iteration += 1
        previous_sse = current_sse
    return cluster_labels, centers, current_sse, iteration


In [ ]:
labels_euclidean, _, sse_euclidean, iterations_euclidean = kmeans_run_full_iterations(samples, num_clusters, 'euclidean')
print("Euclidean SSE:", sse_euclidean)
print("Euclidean Iterations:", iterations_euclidean)

labels_cosine, _, sse_cosine, iterations_cosine = kmeans_run_full_iterations(samples, num_clusters, 'cosine')
print("Cosine SSE:", sse_cosine)
print("Cosine Iterations:", iterations_cosine)

labels_jaccard, _, sse_jaccard, iterations_jaccard = kmeans_run_full_iterations(samples, num_clusters, 'jaccard')
print("Jaccard SSE:", sse_jaccard)
print("Jaccard Iterations:", iterations_jaccard)


Euclidean SSE: 15638058.903263554
Euclidean Iterations: 200
Cosine SSE: 2464.41576026472
Cosine Iterations: 200


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Jaccard SSE: 10000.0
Jaccard Iterations: 200
